In [99]:
#to load file in .csv format and to add 'city' column
import pandas as pd
df=pd.read_csv('/Users/chidam/Desktop/Singapore_after_editing.csv', encoding='latin', sep="\t", header=None)
df['Events'], df['Date'], df['Description'] = df[0].str.split(';', 2).str
del(df[0])
df=df.dropna()
df['City']='Singapore'
df.head()


,Events,Date,Description,City
0,Kreta Ayer Wet Market,year_round,"Singapore Located in Chinatown, both locals a...",Singapore
1,CHIJMES,year_round,A historical convent known for taking in bad ...,Singapore
2,St Joseph's Church,year_round,The first Catholic church in Singapore is a N...,Singapore
3,food heaven,year_round,great experience to live is enjoying local f...,Singapore
4,Marina Bay Sands,year_round,"the Merlion park, Esplanade, the Flyer, the s...",Singapore


In [32]:
#to append dataframe to a csv file:
with open('/Users/chidam/Desktop/training_data.csv', 'a') as f:
    df.to_csv(f, index=False, header=False)

In [245]:
df_train=pd.read_csv('/Users/chidam/Desktop/training_data.csv')
df_train.head()


,Events,Date,Description,City
0,Mevlevi Order of Sufi whirling dervishes,year-round,Go to a whirling dervish show the dervishes s...,Istanbul
1,Eat a kumpir,year-round,a kumpir is a loaded baked potato the server ...,Istanbul
2,Get Hamam experience,year-round,"if you are ready to shed your inhibitions, th...",Istanbul
3,Ride the red street car,year-round,one way to see Istiklal Avenue without having...,Istanbul
4,Walk across Galata Bridge,year-round,you could very well ride the light rail acros...,Istanbul


In [194]:
#trying topic modeling on training data_Set
documents=df_train['Description']

In [195]:
#show me the unique events you have for a city: pick any city
result_favorite=df_train.loc[df_train['City'] == 'San Francisco', 'Events']
result_favorite

248                                   Bar Crudo
249                                Turtle Tower
250                                      Souvla
251                     Duc Loi/Mission Banh Mi
252                           Saigon Sandwiches
253                        El Gallo Giro Truck 
254                   Chinese New Years Parade
255                     Cherry Blossom Festival
256                                Pride Parade
257                         Alice in Winterland
258                      Outside Lands Festival
259                          Folsom Street Fair
260                          Castro Street Fair
261                           Light Up the City
262                               SF Sketchfest
263                              Lunar New Year
264                                SF IndieFest
265                       Winter Music Festival
266                     St Patricks Day Parade
267                     Cherry Blossom Festival
268                               Cinco 

In [268]:
# NMF is able to use tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import numpy as np
import random 
random.seed(13)

tfidf_vectorizer = TfidfVectorizer(max_df=0.85, max_features=45000, min_df=2, use_idf=True, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)    
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
        

In [269]:
#fitting tfidf to nmf model and printing the associated words with every cluster/topic
nmf = NMF(n_components=10, random_state=1, alpha=.1).fit(tfidf)
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(nmf.components_):
        print ("Topic %d:" % (topic_idx))
        
        print (" ".join([tfidf_feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
        #topic.argsort()[:-no_top_words - 1:-1] is to display the top 10 words for each topic - i guess it's like from -9 to -1 like reverse
        

display_topics(nmf, tfidf_feature_names, no_top_words=10)


Topic 0:
bar just restaurant like coffee seoul cafe little great good
Topic 1:
festival annual film multicultural international performances dance rock jazz held
Topic 2:
singapore marina multiple event bay celebrated venues chinese celebration year
Topic 3:
market street night food taipei local stalls seafood selling district
Topic 4:
music electronic free classical world live culture dance pop films
Topic 5:
art museum street theatre film galleries arts gallery artists collection
Topic 6:
city dubai year new world biggest parade events day held
Topic 7:
tel aviv day dance jazz place performances jewish israel avivs
Topic 8:
park garden island beautiful parks central gardens escape secret greenery
Topic 9:
sydney harbour tower world sea tallest eye feet building skywalk


In [270]:
nmf.reconstruction_err_ 

29.148171482548552

In [271]:
#get the top 5 popular events in every cluster
nmf_embedding = nmf.transform(tfidf)
nmf_embedding = (nmf_embedding - nmf_embedding.mean(axis=0))/nmf_embedding.std(axis=0)
top_idx = np.argsort(nmf_embedding,axis=0)[-5:]

In [285]:
#associating the clusters with event type and city name
def event_city(g):
    c=0
    for idxs in g.T:
        print("\nTopic {}:".format(c,))
            print("Event:{}, City:{}".format(df_train.iloc[idx]['Events'], df_train.iloc[idx]['City']))
        c += 1

event_city(top_idx)        



Topic 0:
Event: café-hopping at Garosu-gil, City:Seoul
Event:The Library, City:Singapore
Event:Eat Tafelspitz at Plachuttas, City:Vienna
Event:Foodhallen, City:Amsterdam
Event:Walking tours, City:Munich

Topic 1:
Event:Haarlem Jazz, City:Amsterdam
Event:Danube Carnival, City:Budapest
Event:Viennale, City:Vienna
Event:Voice Mania, City:Vienna
Event:Wean Hean, City:Vienna

Topic 2:
Event:Singapore Food , City:Singapore
Event:River Hongbao, City:Singapore
Event:Hari Raya Haji, City:Singapore
Event:Thaipusam, City:Singapore
Event:Vesak Day, City:Singapore

Topic 3:
Event:Lehua Night Market, City:Taipei
Event:Tonghua Night Market, City:Taipei
Event:Ningxia Night Market at, City:Taipei
Event:Shilin Night Market, City:Taipei
Event:Raohe Street Night Market, City:Taipei

Topic 4:
Event:Zamárdi, City:Budapest
Event:Africa Days Vienna, City:Vienna
Event:Electric Spring, City:Vienna
Event:Sziget Festival, City:Budapest
Event:Jewish Cultural Festival, City:Budapest

Topic 5:
Event:Trick Eye Museu